In [1]:
from torchvision import transforms

net1_data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(64),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Grayscale(),
        transforms.ToTensor()
        
    ]),
    'val': transforms.Compose([
        transforms.Resize(64),
        transforms.Grayscale(),
        transforms.ToTensor(),
    ]),
    
    'test': transforms.Compose([
        transforms.Resize(64),
        transforms.Grayscale(),
        transforms.ToTensor(),
    ]),
}

In [2]:

import sys
import os
sys.path.append(os.path.abspath('../../../'))
from detection.lib.model.ImageROI import ImageROI
from PIL import Image
import numpy as np
import torch

# book_data_dir = '/home/rahul/data/printed_with_ids_harsh_filter/books-preprocessed-images'
# img_loc = book_data_dir + '/Dryden/positive/307.png'
# Testing with image of dimensions 1000x1381

img_loc = '/home/rahul/data/printed_with_ids_harsh_filter/books-preprocessed-images/Allestree/positive/1307.png'
pilimg = Image.open(img_loc)
w, h = pilimg.size
imgrois = [ImageROI(0, 0, w, h)]
imgarr = np.asarray(pilimg)

from model_utils import create_model_architecture
from bounding_box_classification import get_pos_rois
# net1 = Net1()
net1 = create_model_architecture('net1', use_gpu = False)
net1.load_state_dict(torch.load('models/net1-IAM-1.pt'))

# Modify this as necessary
# imgrois = [ImageROI(0, 0, 1000, 700), ImageROI(0, 750, 1000, 500)]
net1.train(False)
with torch.set_grad_enabled(False):
    pos_rois = get_pos_rois(net1,
                            (img_loc, imgrois),
                            model_transform = net1_data_transforms['test'],
                            model_input_size = (64,64),
                            stride=(32, 32))

# for rois in pos_rois:
#     for roi in rois:
#         print(roi.toString())
        
all_rois = [roi for rois in pos_rois for roi in rois]
    
print('number of pos ROIs found', len(all_rois))

# from detection.lib.ImgProcessor import ImgProcessor
from detection.lib.utils.Utils import plt_img
# plot the rois and plot only certain information according to the set
# imgProcessor = ImgProcessor()
# img = imgProcessor.loadImage(inFilename=img_loc)
# roi_level_set = set([1])
plt_img(imgarr, rois=[(all_rois, 'r')])#, roi_level_set=roi_level_set)

Creating Net1.
<class 'torch.Tensor'>
number of pos ROIs found 346
x:960	y:32	width:64	height:64	text:	confidence:-0.09819002449512482	level:2	isAnnotation:True
x:960	y:64	width:64	height:64	text:	confidence:-0.09819002449512482	level:2	isAnnotation:True
x:960	y:160	width:64	height:64	text:	confidence:-0.09819002449512482	level:2	isAnnotation:True
x:960	y:192	width:64	height:64	text:	confidence:-0.2849978804588318	level:2	isAnnotation:True
x:960	y:224	width:64	height:64	text:	confidence:-0.36542436480522156	level:2	isAnnotation:True
x:960	y:256	width:64	height:64	text:	confidence:-0.20056644082069397	level:2	isAnnotation:True
x:960	y:288	width:64	height:64	text:	confidence:-0.6336069703102112	level:2	isAnnotation:True
x:960	y:320	width:64	height:64	text:	confidence:-0.3299109637737274	level:2	isAnnotation:True
x:960	y:352	width:64	height:64	text:	confidence:-0.09819002449512482	level:2	isAnnotation:True
x:960	y:384	width:64	height:64	text:	confidence:-0.6531566381454468	level:2	isAnnot

x:1120	y:896	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1120	y:928	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:0	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:32	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:64	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:96	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:128	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:160	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:192	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:224	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:256	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:288	width:64	height:64	text:	confidence:0.0	level:2	isAnnotation:True
x:1152	y:320	width:64	height:64	text:	confidence:0.0	leve